# Effective Stiffness of Fiber Composite



## Introduction

This example demonstrates the use of the homogenization model from pyMKS on a set of fiber-like structures.  These structures are simulated to emulate fiber-reinforced polymer samples.  For a summary of homogenization theory and its use with effective stiffness properties please see the [Effective Siffness example](http://materialsinnovation.github.io/pymks/rst/stress_homogenization_2D.html).  This example will first generate a series of random microstructures with various fiber lengths and volume fraction.  The ability to vary the volume fraction is a new functionality of this example.  Then the generated stuctures will be used to calibrate and test the model based on simulated effective stress values.  Finally we will show that the simulated response compare favorably with those generated by the model.  

## Generating Structures

These first lines inport important packages that will be used to run pymks.  

In [1]:
# %matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt


Now we are defining the parameters which we will use to create the microstructures.  `n_samples` will determine how many microstructures of a particular volume fraction we want to create.  `size` determines the number of pixels we want to be included in the microstructure.  We will define the material properties to be used in the finite element in `elastic_modulus`, `poissons_ratio` and `macro_strain`.  `n_phases` and `grain_size` will determine the physical characteristics of the microstructure.  We are using a high aspect ratio in creating our microstructures to simulate fiber-like structures.  The `volume_fraction` variable will be used to vary the fraction of each phase.  The sum of the volume fractions must be equal to 1.  The `percent_variance` variable introduces some variation in the volume fraction up to the specified percentage.

In [2]:
sample_size = 200
n_samples = 6 * [sample_size]
size = (101, 101)
elastic_modulus = (1.3, 75)
poissons_ratio = (0.42, .22)
macro_strain = 0.001
n_phases = 2
grain_size = [(40, 2), (10, 2), (2, 40), (2, 10), (2, 30), (30, 2)]
v_frac = [(0.8, 0.2), (0.7, 0.3), (0.6, 0.4), (0.5, 0.5), (0.3, 0.7), (0.4, 0.6)]
per_ch = 0.1
seed=10

In [3]:
# from pymks.datasets.elastic_FE_simulation import ElasticFESimulation

# sim = ElasticFESimulation(poissons_ratio=poissons_ratio,
#                           elastic_modulus=elastic_modulus)

Now we will create the microstructures and generate their responses using the `make_elastic_stress_random` function from pyMKS.  Four datasets are created to create the four different volume fractions that we are simulating.  Then the datasets are combined into one variable.   The volume fractions are listed in the variable `v_frac`.  Variation around the specified volume fraction can be obtained by varying `per_ch`.  The variation is randomly generated according a uniform distribution around the specified volume fraction.

In [4]:
# from pymks.datasets import make_microstructure

# X = make_microstructure(size=(25, 25))
# X.shape

In [5]:
# X = np.concatenate([make_microstructure(n_samples=sample, size=size,
#                                         n_phases=2,
#                                         grain_size=gs, seed=seed,
#                                         volume_fraction=vf,
#                                         percent_variance=per_ch)
#                     for vf, gs, sample in zip(v_frac,
#                                               grain_size, n_samples)])

# print X.shape


In [6]:
# sim.run(X)
# local_strain = sim.response
# print local_strain.shape



Now we are going to print out a few microstructres to look at how the fiber length, orientation and volume fraction are varied.  

In [7]:
# from pymks import PrimitiveBasis

# p_basis = PrimitiveBasis(2)

# X_ = p_basis.discretize(X)
# index = tuple([None for i in range(len(size) + 1)]) + (slice(None),)
# modulus = np.sum(X_ * np.array(elastic_modulus)[index], axis=-1)
# y_stress_field = local_strain * modulus
# effective_stress = np.average(y_stress_field.reshape(len(y_stress_field), -1), axis=1)
# print effective_stress.shape

In [8]:
import cPickle as pickle
# pickle.dump((X, effective_stress), open('fiber_data.pkl', 'wb'))
dataset, stresses = pickle.load(open('fiber_data.pkl', 'rb'))


In [189]:
moduli = stresses / macro_strain / 100

In [22]:
from pymks.tools import draw_microstructures
examples = dataset[::sample_size]
draw_microstructures(examples)


## 2-Point Statistics

In [190]:
from pymks.stats import correlate
from pymks import PrimitiveBasis


p_basis = PrimitiveBasis(n_states=2, domain=[0, 1])
data_stats = correlate(dataset[::sample_size], p_basis, correlations=[(0, 0)], periodic_axes=[0, 1])

In [191]:
from pymks.tools import draw_correlations

print data_stats.shape
draw_correlations(np.transpose(data_stats, (3, 1, 2, 0))[0], correlations=[('fiber', 'fiber')] * 6)

(6L, 101L, 101L, 1L)


## Creating the Model

Next we are going to initiate the model. The MKSHomogenizationModel takes in microstructures and runs two-point statistics on them to get a statistical representation of the microstructures.  An expalnation of the use of two-point statistics can be found in the [Checkerboard Microstructure Example](http://materialsinnovation.github.io/pymks/rst/checker_board.html).  Then the model uses PCA and regression models to create a linkage between the calcualted properties and structures.  
Here we simply initiate the model.  

In [203]:
from pymks import MKSHomogenizationModel

model = MKSHomogenizationModel(basis=p_basis, correlations=[(1, 1), (0, 1)], periodic_axes=[0, 1])


Now we are going to split our data into testing and training segments so we can test and see if our model can accurately predict the effective stress. 


In [204]:
from sklearn.cross_validation import train_test_split


flat_shape = (dataset.shape[0],) + (dataset[0].size,)
data_train, data_test, moduli_train, moduli_test = train_test_split(
    dataset.reshape(flat_shape), moduli, test_size=0.2, random_state=3)


We will use sklearn's GridSearchCV to optimize the `n_components` and `degree` for our model. Let's search over the range of 1st order to 3rd order polynomial for `degree` and 2 to 7 principal components for `n_components`.

In [205]:
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)

params_to_tune = {'degree': np.arange(1, 7), 'n_components': np.arange(2, 8)}
fit_params = {'size': dataset[0].shape}
gs = GridSearchCV(model, params_to_tune, fit_params=fit_params, scoring=mse_scorer).fit(data_train, moduli_train)


Let's take a look at the results.

In [ ]:
print('Order of Polynomial'), (gs.best_estimator_.degree)
print('Number of Components'), (gs.best_estimator_.n_components)
print('Score'), (gs.score(data_test, stress_test))


Order of Polynomial 4
Number of Components 4
Score -160.806302603


In [ ]:
from pymks.tools import draw_gridscores_matrix

draw_gridscores_matrix(gs, ['n_components', 'degree'], score_label='MSE',
                       param_labels=['Number of Components', 'Order of Polynomial'])


In [196]:
from pymks.tools import draw_gridscores

# gs_deg_1 = [x for x in gs.grid_scores_ \
#             if x.parameters['degree'] == 1]
_n = 4
gs_deg_2 = [x for x in gs.grid_scores_ \
            if x.parameters['degree'] == 2][:_n]
gs_deg_3 = [x for x in gs.grid_scores_ \
            if x.parameters['degree'] == 3][:_n]
gs_deg_4 = [x for x in gs.grid_scores_ \
            if x.parameters['degree'] == 4][:_n]
gs_deg_5 = [x for x in gs.grid_scores_ \
            if x.parameters['degree'] == 5][:_n]
gs_deg_6 = [x for x in gs.grid_scores_ \
            if x.parameters['degree'] == 6][:_n]

draw_gridscores([gs_deg_2, gs_deg_3, gs_deg_4, gs_deg_5, gs_deg_6], 'n_components', 
                data_labels=['2nd Order', '3rd Order', '4th Order', '5', '6'],
                param_label='Number of Components', score_label='MSE')


Our best model was found to have `degree` equal to 3 and `n_components` equal to 5. Let's go ahead and use it.

In [197]:
model = gs.best_estimator_
# model.n_components = 7
model.fit(data_train.reshape((data_train.shape[0],) + dataset.shape[1:]), moduli_train)

### Structures in PCA space

Now we want to draw how the samples are spread out in PCA space and look at how the testing and training data line up.  


In [198]:
from pymks.tools import draw_component_variance

draw_component_variance(model.dimension_reducer.explained_variance_ratio_)

In [184]:
from pymks.tools import draw_components

print 2 * 101 * 101
print model.dimension_reducer.components_.reshape((7, 202 * 101))

draw_components(model.dimension_reducer.components_.reshape((7, 202 * 101))[0])

20402
[[ 0.00700901 -0.00700901  0.00701734 ..., -0.00701734  0.00700901
  -0.00700901]
 [-0.00028885  0.0002887   0.00025726 ..., -0.00025739 -0.00028875
   0.00028876]
 [ 0.0004406  -0.00043145 -0.00054912 ...,  0.00056838  0.00043921
  -0.00044048]
 ..., 
 [ 0.02276175 -0.00073187  0.01734732 ..., -0.00730536  0.00542154
  -0.00721103]
 [ 0.02518966  0.00509389  0.00797646 ...,  0.00024712  0.00304061
  -0.00110592]
 [ 0.01298807  0.00159487  0.02885102 ..., -0.00417512  0.00768894
  -0.0076502 ]]


KeyboardInterrupt: 

In [ ]:
from pymks.tools import draw_components_scatter

draw_component_variance


In [199]:
from pymks.tools import draw_components_scatter


moduli_predict = model.predict(data_test.reshape((data_test.shape[0],) + dataset.shape[1:]))
draw_components_scatter([model.reduced_fit_data[:, :3],
                         model.reduced_predict_data[:, :3]],
                        ['Training Data', 'Testing Data'],
                        legend_outside=True)


It looks like there is pretty good agreement between the testing and the training data.  We can also see that the four different fiber sizes are seperated in the PC space.  

### Draw Goodness of fit

Now we are going to look at how well our model predicts the properties of the structures.  The calculated properties will be plotted against the properties generated by the model.  We should see a linear realtionship with a slope of 1.   

In [200]:
from pymks.tools import draw_goodness_of_fit

_moduli_train = model.predict(data_train.reshape((data_train.shape[0],) + dataset.shape[1:]))

fit_data = np.array([moduli_train, _moduli_train])
pred_data = np.array([moduli_test, moduli_predict])
draw_goodness_of_fit(fit_data, pred_data, ['Training Data', 'Testing Data'])


### Percent Error 

In [202]:
pred_percent_error = 100 * np.abs(moduli_predict - moduli_test) / moduli_test
train_percent_error = 100 * np.abs(_moduli_train - moduli_train) / moduli_train
pred_absolute_error = np.abs(stress_predict - stress_test)
train_absolute_error = np.abs(stresses - stress_train)
print 'Mean Predicted Percent Error', np.mean(pred_percent_error)
print 'Mean Training Percent Error', np.mean(train_percent_error)


Mean Predicted Percent Error 2.55770320384
Mean Training Percent Error 2.14920482809


In [162]:
plt.plot(stresses.flatten(), train_absolute_error.flatten(), 'o', color='#1a9850', label='Training Dataset')
plt.plot(stress_test.flatten(), pred_absolute_error.flatten(), 'o', color='#f46d43', label='Testing Dataset')
plt.xlabel('Effective Stress (MPa)', fontsize=15)
plt.ylabel('Abosolute Error (MPa)', fontsize=15)
plt.legend(fontsize=12, loc=2)
plt.title('Absolute Error', fontsize=20)
plt.ylim(-0.05, np.max(train_absolute_error) + 0.3)
plt.show()

In [34]:
# plt.hist(stresses.flatten())
# plt.show()

Yay! There is a good corrolation between the FE results and those predicted by our linkage.  